In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from src.extra.model import ModelDistribution
from src.extra.pprint import print_model
from src.core.core import reset_model

from src.greedy.greedy import greedy_algorithm
from src.greedy.resource_allocation_policy import SumPercentage
from src.greedy.server_selection_policy import SumResources
from src.greedy.task_prioritisation import UtilityDeadlinePerResource

from src.optimal.fixed_optimal import fixed_optimal
from src.core.fixed_task import FixedTask, SumSpeedPowFixedPrioritisation

In [3]:
def print_results(result, tasks, servers):
    print(f'Social welfare: {result.social_welfare:.3f}, Percentage tasks allocated: {result.percentage_tasks_allocated:.3f}\n')
    for server in servers:
        storage_usage = sum(task.required_storage for task in server.allocated_tasks) / server.storage_capacity
        computational_usage = sum(task.compute_speed for task in server.allocated_tasks) / server.computation_capacity
        bandwidth_usage = sum(task.loading_speed + task.sending_speed for task in server.allocated_tasks) / server.bandwidth_capacity
        print(f'{server.name} resouce usage - Stor: {storage_usage:.3f}, Comp: {computational_usage:.3f}, Band: {bandwidth_usage:.3f}')
    
    avg_loading_speed = np.mean([task.loading_speed for task in tasks if task.running_server])
    avg_compute_speed = np.mean([task.compute_speed for task in tasks if task.running_server])
    avg_sending_speed = np.mean([task.sending_speed for task in tasks if task.running_server])
    print(f'\nAvg loading speed: {avg_loading_speed}, compute speed: {avg_compute_speed}, sending speed: {avg_sending_speed}')
    avg_loading_time = np.mean([task.required_storage / task.loading_speed for task in tasks if task.running_server])
    avg_compute_time = np.mean([task.required_computation / task.compute_speed for task in tasks if task.running_server])
    avg_sending_time = np.mean([task.required_results_data / task.sending_speed for task in tasks if task.running_server])
    print(f'Avg loading time: {avg_loading_time}, compute time: {avg_compute_time}, sending time: {avg_sending_time}')

In [4]:
alibaba_model = ModelDistribution('../realistic.mdl', num_tasks=100, num_servers=10)

In [5]:
alibaba_model.storage_scaling = 150
alibaba_model.computational_scaling = 0.25
alibaba_model.results_data_scaling = 1

In [6]:
tasks, servers = alibaba_model.generate()

In [7]:
result = greedy_algorithm(tasks, servers, UtilityDeadlinePerResource(), SumResources(), SumPercentage())
print_results(result, tasks, servers)

Social welfare: 415.820, Percentage tasks allocated: 0.720

medium 0 resouce usage - Stor: 1.000, Comp: 0.016, Band: 0.100
2xlarge 1 resouce usage - Stor: 0.984, Comp: 0.042, Band: 0.088
large 2 resouce usage - Stor: 0.987, Comp: 0.028, Band: 0.127
2xlarge 3 resouce usage - Stor: 0.992, Comp: 0.065, Band: 0.112
medium 4 resouce usage - Stor: 0.980, Comp: 0.016, Band: 0.040
medium 5 resouce usage - Stor: 1.000, Comp: 0.012, Band: 0.090
2xlarge 6 resouce usage - Stor: 1.000, Comp: 0.134, Band: 0.148
2xlarge 7 resouce usage - Stor: 1.000, Comp: 0.050, Band: 0.104
xlarge 8 resouce usage - Stor: 0.990, Comp: 0.052, Band: 0.175
2xlarge 9 resouce usage - Stor: 1.000, Comp: 0.115, Band: 0.116

Avg loading speed: 1.8194444444444444, compute speed: 9.791666666666666, sending speed: 1.2222222222222223
Avg loading time: 8.00896686159844, compute time: 28.157819748420795, sending time: 2.949404761904762


In [8]:
reset_model(tasks, servers)
fixed_tasks = [FixedTask(task, SumSpeedPowFixedPrioritisation()) for task in tasks]
foreknowledge_fixed_tasks = [FixedTask(task, SumSpeedPowFixedPrioritisation(), resource_foreknowledge=True) 
                             for task in tasks]

In [9]:
reset_model(tasks, servers)
result = fixed_optimal(fixed_tasks, servers, time_limit=4)
print_results(result, fixed_tasks, servers)

Social welfare: 270.980, Percentage tasks allocated: 0.230

medium 0 resouce usage - Stor: 0.900, Comp: 0.100, Band: 0.230
2xlarge 1 resouce usage - Stor: 0.952, Comp: 0.067, Band: 0.056
large 2 resouce usage - Stor: 1.000, Comp: 0.100, Band: 0.413
2xlarge 3 resouce usage - Stor: 0.952, Comp: 0.067, Band: 0.272
medium 4 resouce usage - Stor: 0.900, Comp: 0.100, Band: 0.840
medium 5 resouce usage - Stor: 0.940, Comp: 0.100, Band: 0.540
2xlarge 6 resouce usage - Stor: 0.952, Comp: 0.050, Band: 0.196
2xlarge 7 resouce usage - Stor: 0.944, Comp: 0.067, Band: 0.064
xlarge 8 resouce usage - Stor: 0.980, Comp: 0.100, Band: 0.365
2xlarge 9 resouce usage - Stor: 0.960, Comp: 0.067, Band: 0.228

Avg loading speed: 17.652173913043477, compute speed: 34.78260869565217, sending speed: 4.086956521739131
Avg loading time: 5.153966999184391, compute time: 28.565217391304348, sending time: 1.926557500470544


In [10]:
reset_model(tasks, servers)
result = fixed_optimal(foreknowledge_fixed_tasks, servers, time_limit=4)
print_results(result, foreknowledge_fixed_tasks, servers)

Social welfare: 453.420, Percentage tasks allocated: 0.750

medium 0 resouce usage - Stor: 0.980, Comp: 0.138, Band: 0.520
2xlarge 1 resouce usage - Stor: 0.992, Comp: 0.055, Band: 0.296
large 2 resouce usage - Stor: 0.987, Comp: 0.161, Band: 0.613
2xlarge 3 resouce usage - Stor: 1.000, Comp: 0.023, Band: 0.192
medium 4 resouce usage - Stor: 1.000, Comp: 0.038, Band: 0.180
medium 5 resouce usage - Stor: 0.960, Comp: 0.100, Band: 0.280
2xlarge 6 resouce usage - Stor: 1.000, Comp: 0.067, Band: 0.324
2xlarge 7 resouce usage - Stor: 0.992, Comp: 0.040, Band: 0.256
xlarge 8 resouce usage - Stor: 0.990, Comp: 0.163, Band: 0.560
2xlarge 9 resouce usage - Stor: 1.000, Comp: 0.057, Band: 0.332

Avg loading speed: 5.013333333333334, compute speed: 10.48, sending speed: 3.68
Avg loading time: 4.969093393700471, compute time: 28.56369500708914, sending time: 1.6534853041695143
